In [1]:
import numpy as np
from importlib import reload

from icedef import timestepper, drift, metocean, iceberg, interpolate, tools

In [2]:
time_stepper = timestepper.euler
interpolator = interpolate.UniformRegularLinearInterpolator
drift_model = drift.DynamicDriftModel
ocean_model = 'ECMWF'
atmosphere_model = 'ECMWF'

In [3]:
reload(metocean)  

<module 'icedef.metocean' from '/home/evankielley/IceDEF/icedef/metocean.py'>

In [4]:
start_date = np.datetime64('2015-04-23')
end_date = start_date + np.timedelta64(1, 'D')
date_bounds = start_date, end_date

In [5]:
ocean = metocean.Ocean(date_bounds, model=ocean_model, interpolator=interpolator)
atmosphere = metocean.Atmosphere(date_bounds, model=atmosphere_model, interpolator=interpolator)

In [6]:
t = start_date + np.timedelta64(1, 'h')
lat = 50
lon = -50
point = (t, lat, lon)
ocean.current.interpolate(point), atmosphere.wind.interpolate(point)

(array([ 0.08331566, -0.18555277]), array([-1.73470052, -2.72474162]))

In [7]:
iceberg_start_time = start_date + np.timedelta64(1, 'h')
iceberg_start_position = 50, -50
iceberg_start_velocity = 0, 0
iceberg_ = iceberg.quickstart(iceberg_start_time, iceberg_start_position, iceberg_start_velocity)

In [8]:
drift_model_kwargs = {
    'form_drag_coefficient_in_air': iceberg_.FORM_DRAG_COEFFICIENT_IN_AIR,
    'form_drag_coefficient_in_water': iceberg_.FORM_DRAG_COEFFICIENT_IN_WATER,
    'skin_drag_coefficient_in_air': iceberg_.SKIN_DRAG_COEFFICIENT_IN_AIR,
    'skin_drag_coefficient_in_water': iceberg_.SKIN_DRAG_COEFFICIENT_IN_WATER,
    'sail_area': iceberg_.geometry.sail_area,
    'keel_area': iceberg_.geometry.keel_area,
    'top_area': iceberg_.geometry.top_area,
    'bottom_area': iceberg_.geometry.bottom_area,
    'mass': iceberg_.geometry.mass,
}

drift_model = drift_model()
drift_function = drift_model.drift

In [11]:
time_step = np.timedelta64(300, 's')
dt = time_step.item().total_seconds()

simulation_start_time = iceberg_start_time
simulation_end_time = simulation_start_time + np.timedelta64(1, 'h')

time = simulation_start_time
iceberg_position = iceberg_start_position
iceberg_velocity = iceberg_start_velocity

while time <= simulation_end_time:
    
    point = time, iceberg_position[0], iceberg_position[1]
    
    drift_function_kwargs = {
        'current_velocity': ocean.current.interpolate(point),
        'mean_current_velocity': ocean.current.interpolate(point),
        'mean_current_acceleration': (0, 0),
        'wind_velocity': atmosphere.wind.interpolate(point),
        'latitude': iceberg_start_position[0],
    }
    
    dx, dy, dvx, dvy = time_stepper(drift_model.drift, dt, iceberg_velocity, **drift_function_kwargs)
    
    iceberg_eastward_velocity, iceberg_northward_velocity = iceberg_velocity
    iceberg_eastward_velocity += dvx
    iceberg_northward_velocity += dvy
    iceberg_velocity = iceberg_eastward_velocity, iceberg_northward_velocity
    iceberg_latitude, iceberg_longitude = iceberg_position
    iceberg_latitude += tools.dy_to_dlat(dy)
    iceberg_longitude += tools.dx_to_dlon(dx, iceberg_latitude)
    iceberg_position = iceberg_latitude, iceberg_longitude
    
    print(iceberg_position)
    
    time += time_step

(50.0, -50.0)
(49.999861272620954, -49.99989305021428)
(49.99964800515515, -49.999732651438045)
(49.9993856792163, -49.99954002937828)
(49.999087680361285, -49.999326329382626)
(49.99876208939809, -49.99909827751343)
(49.99841422786418, -49.99886030184033)
(49.998047819418716, -49.99861550016457)
(49.99766579762618, -49.99836638819771)
(49.99727036502455, -49.99811485940936)
(49.99686322563669, -49.99786238364858)
(49.996445730762964, -49.997610131694344)
(49.996018974570845, -49.997359056891035)
